# **Python for Automation in Bioimage Analysis Workflows**

Manual analysis is often time-consuming, error-prone, and unscalable. 
Python, with its extensive ecosystem of scientific libraries, provides powerful tools to automate bioimage analysis pipelines,  scalability, improving efficiency, reproducibility, and accuracy.
By leveraging specialized libraries, researchers can streamline workflows, extract meaningful insights, and accelerate discoveries in microscopy-based studies.

## **Examples of tasks in a bioimage analysis workflow that can be automated:**

**1. Preprocessing Automation**

Batch processing of images using loops or parallelization with multiprocessing.

Standardizing image formats and resolutions.

Applying denoising and contrast enhancement techniques automatically.

**2. Segmentation and Feature Extraction**

Using thresholding, edge detection, or deep learning-based segmentation.

Automating cell counting and morphology quantification.

Storing extracted features in structured datasets.

**3. Machine Learning for Classification**

Training models to classify different cell types or conditions.

Using transfer learning for enhanced accuracy.

Deploying automated classification in real-time analysis workflows.

**4. Results Visualization and Reporting**

Generating automated plots and reports using matplotlib and seaborn.

Exporting results in structured formats (CSV, JSON, or PDF).

In [ ]:
#import libraries
import os
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
from skimage.io import  imshow, imsave 
from skimage.measure import regionprops_table
import matplotlib
import pandas as pd
import napari
import re


# Let´s automate segmentation and quantification of several images in a folder

#  A few words about the  Dataset we will be exploring
Link for downloading the dataset https://bbbc.broadinstitute.org/BBBC014
This 96-well plate has images of cytoplasm to nucleus translocation of the transcription factor NFκB in MCF7 (human breast adenocarcinoma cell line) and A549 (human alveolar basal epithelial) cells in response to TNFα concentration.

For this notebook, we only worked with the MCF7 cells (located in wells A-D)

Images
Images are at 10x objective magnification. The plate was acquired at Vitra Bioscience on the CellCard reader. For each well there is one field with two images: a nuclear counterstain (DAPI) image and a signal stain (FITC) image. Image size is 1360 x 1024 pixels. Images are in 8-bit BMP format.

Recommended citation
"We used image set BBBC014v1 provided by Ilya Ravkin, available from the Broad Bioimage Benchmark Collection [Ljosa et al., Nature Methods, 2012]."

Copyright
Creative Commons LicenseThe SBS Bioimage CNT images are licensed under a Creative Commons Attribution 3.0 Unported License by Ilya Ravkin.

In [ ]:
os.chdir('/Users/user/path_to_your_working_directory') #if using 
print(os.getcwd())

# Automation for Cellpose segmentation

In [ ]:

nucleo_files = np.sort(glob.glob('image_folder/*Channel 2*.tiff'))
#make label folder
!mkdir BBC014_label_folder
# Define the label_folder
label_folder = 'label_folder/' # replace with your actual folder path

# load the cellpose model outside the loop
model = models.Cellpose(gpu=True, model_type='nuclei') #if you do not have GPU on your computer,  replace True for False in the code
channels = [0,0] # This means we are processing single channel greyscale images for cellpose


for i in nucleo_files:
    nucleo_ch1 = plt.imread(i)
    print("Processing image:", i)
    # apply cellpose model
    label_image, *_ = model.eval(nucleo_ch1, flow_threshold=0.8, diameter=10, channels=channels)
    
    #save label images in a folder
    #save label_image as tiff
    imsave(os.path.join(label_folder,  f"{os.path.splitext(os.path.basename(i))[0]}.tiff"), label_image)
   

In [ ]:
nucleo_files

## Visualize segmentation results using Napari

Learn more about Napari here https://napari.org/stable/

In [ ]:
raw_images_path = Path('image_folder/')
label_images_path = Path('label_folder/')
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(raw_images_path.glob("*Channel 2*.tiff")), stack=True)
viewer.open(sorted(raw_images_path.glob("*Channel 1*.tiff")), stack=True)
viewer.open(sorted(label_images_path.glob("*Channel*.tiff")), stack=True)
cellpose_label = viewer.layers[2].data
viewer.add_labels(cellpose_label, visible=True)


# Automation for measurements

In [ ]:
#

results =[]
target_files = np.sort(glob.glob('image_folder/*Channel 1*.tiff'))
label_files = np.sort(glob.glob('label_folder/*Channel*.tiff'))

properties = 'label', 'area', 'mean_intensity'

for i in range(len(label_files)):
    label_images = plt.imread(label_files[i])
    tnf = plt.imread(target_files[i])
    
    
    data = regionprops_table(label_images, intensity_image=tnf,  properties =properties)
    

    df = pd.DataFrame(data)

    df["file_name"]= target_files[i].split('\\')[1]
    df["condition"]= re.findall('.+ .+-.+-.+-(.+)-.+.tiff',target_files[i].split('\\')[1])[0]
    df['replicate']= re.findall('.+ .+-.+-(.+)-.+-.+.tiff',target_files[i].split('\\')[1])[0]
    #Channel 1-01-A-01-00.Bmp
    
    
    results.append(df)

In [ ]:
all_results = pd.concat(results)


In [ ]:
all_results.tail(10)

## Let's do some data visualization 

In [ ]:

import seaborn as sns
sns.boxplot(data=all_results, x='condition', y='area', 
              hue='replicate')

In [ ]:
sns.jointplot(data=all_results, x='mean_intensity', y='area', 
              hue='condition')